# Lesson 4: Introduction to dendrites

## Lesson goals:

1. Discuss challenge from Lesson 3:
 - Devise an objective function for optimization of rheobase.
2. Discuss papers from last week focusing on filtering of synaptic signals from dendrites to soma
3. Explore attenuation of signals from dendrite to soma.
4. Challenge - Reproduce Figures 2b, 3a, and 3c from Magee & Cook

## 1. Discuss challenge from Lesson 3:

### Devise an objective function for optimization of rheobase.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

These last two weeks we used the default compartment size, which models the Hodgkin-Huxley squid axon compartment. But we were only able to tune the input resistance to be ~10 MOhm, and the rheobase to be ~0.5 nA.

Let's change the dimensions of the compartment to resemble a rodent pyramidal neuron, and attempt to tune the input resistance to be ~100 MOhm, and the rheobase to be 0.1 nA.

In [ ]:
soma = h.Section()
soma.L = 20.  # um
soma.diam = 20.  # um
soma.Ra = 100.  # MOhm

soma.insert('hh')

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t)  # record the time stamp
soma_voltage.record(soma(0.5)._ref_v)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
step_current_stim.amp = -0.1  # amplitude in nanoAmps
step_current_stim.dur = 200.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

step_current_rec = h.Vector()
step_current_rec.record(step_current_stim._ref_i)

h.tstop = 600.

Let's store the initial values of each of the ion channel conductances:

In [ ]:
gl_init = soma(0.5).hh.gl
gnabar_init = soma(0.5).hh.gnabar
gkbar_init = soma(0.5).hh.gkbar

print('Initial leak condutance is %.5f S/cm^2' % gl_init)
print('Initial Na condutance is %.5f S/cm^2' % gnabar_init)
print('Initial K condutance is %.5f S/cm^2' % gkbar_init)

First let's optimize the leak conductance to hit a target input resistance of 100 MOhm.

We can re-use some functions from last week:

In [ ]:
def get_input_resistance(t, vm, i, start, stop, window_dur):
    
    baseline_start_index = np.where(t >= start - window_dur)[0][0]
    baseline_end_index = np.where(t >= start)[0][0]
    equil_start_index = np.where(t >= stop - window_dur)[0][0]
    equil_end_index = np.where(t >= stop)[0][0]
    delta_vm = np.abs(np.mean(vm[equil_start_index:equil_end_index]) - np.mean(vm[baseline_start_index:baseline_end_index]))
    delta_i = np.abs(np.mean(i[equil_start_index:equil_end_index]) - np.mean(i[baseline_start_index:baseline_end_index]))
    
    # Ohms = Volts / Amps; MegaOhms = milliVolts / nanoAmps
    input_res = delta_vm / delta_i
    return input_res


def simulate_model_test_input_resistance(plot=False):
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['gl'] = soma(0.5).hh.gl
    
    sim_record['description'] = 'gl: %.2E S/cm^s' % soma(0.5).hh.gl
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    input_resistance = get_input_resistance(sim_record['t'], sim_record['soma_voltage'], sim_record['step_current_rec'], 
                                            start=step_current_stim.delay, stop=step_current_stim.delay + step_current_stim.dur, 
                                            window_dur=10.)
    sim_record['input_resistance'] = input_resistance
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(8, 3))
        axes[0].plot(sim_record['t'], sim_record['step_current_rec'], label='%.3f nA' % (sim_record['stim_amp']))
        axes[0].set_xlabel('Time (ms)')
        axes[0].set_ylabel('Current (nA)')
        axes[0].set_title('Injected current')
        axes[1].plot(sim_record['t'], sim_record['soma_voltage'], label='R_inp: %.3f MOhm' % sim_record['input_resistance'])
        axes[1].set_xlabel('Time (ms)')
        axes[1].set_ylabel('Voltage (mV)')
        axes[1].set_title('Soma membrane potential')
        axes[0].legend(loc='best', frameon=False)
        axes[1].legend(loc='best', frameon=False)
        fig.suptitle(sim_record['description'])
        fig.tight_layout(w_pad=0.9)
    
    return sim_record

There is one change that I like to make to the objective error function. We can enforce bound constrains on the parameters. Some optimization algorithms never choose parameters outside specified bounds. If this is not the case, we can manually exclude bad parameters by simply returning a very large error value.

In [ ]:
def get_input_resistance_objective_error(x, target, tolerance, bounds=None, sim_history=None, plot=False):
    
    # expect x to be an array of length 1. The single element is a value of gl to test.
    gl = x[0]
    
    if bounds is not None:
        for i, (xmin, xmax) in enumerate(bounds):
            if not (xmin <= x[i] <= xmax):
                return 1e9
    
    soma(0.5).hh.gl = gl
    
    sim_record = simulate_model_test_input_resistance(plot=plot)
    sim_record['x'] = np.copy(x)
    
    input_resistance = sim_record['input_resistance']
    error = ((target - input_resistance) / tolerance) ** 2.
    sim_record['input_resistance_error'] = error
    
    if sim_history is not None:
        sim_history.append(sim_record)
    
    return error

Let's make sure our functions work at the initial conditions:

In [ ]:
bounds=[(gl_init / 100., gl_init * 100.)]
target = 100.  # Input resistance in MOhm
tolerance = 1.  # Tolerance in MOhm
x0 = [gl_init]

get_input_resistance_objective_error(x0, target, tolerance, bounds, plot=True)

First we will try a gradient-based algorithm:

In [ ]:
from scipy.optimize import minimize
sim_history = []

result = minimize(get_input_resistance_objective_error, x0, method='L-BFGS-B', bounds=bounds, options={'disp': True}, 
                  args=(target, tolerance, bounds, sim_history))

In [ ]:
print(result)

Let's look at the simulation results using this optimized parameter value:

In [ ]:
get_input_resistance_objective_error(result.x, target, tolerance, bounds, plot=True)

Let's visualize the values of conductance, input resistance, and objective error sampled during optimization:

In [ ]:
x_history = []
input_resistance_history = []
error_history = []
for sim_record in sim_history:
    x_history.append(sim_record['x'][0])
    input_resistance_history.append(sim_record['input_resistance'])
    error_history.append(sim_record['input_resistance_error'])

fig, axes = plt.subplots(1, 2)
axes[0].scatter(x_history, input_resistance_history)
axes[0].set_xlabel('Leak conductance (S/cm^2)')
axes[0].set_ylabel('Input resistance (MOhm)')

axes[1].scatter(x_history, error_history)
axes[1].set_xlabel('Leak conductance (S/cm^2)')
axes[1].set_ylabel('Input resistance objective error')
fig.tight_layout()

Let's store and set this new value of leak conductance and move on to tune the Na and K channel conductances to fit spiking objectives.

In [ ]:
gl_new = result.x[0]
soma(0.5).hh.gl = gl_new

Let's re-use our convenience functions for finding spike times from last week:

In [ ]:
def get_spike_times(t, vm, vm_threshold=0.):
    spike_times = []
    i = 0
    while i < len(t):
        if vm[i] >= vm_threshold:
            spike_times.append(t[i])
            i += 1
            while i < len(t) and vm[i] >= vm_threshold:
                i += 1
        else:
            i += 1
    return np.array(spike_times)

It was also convenient to define a function to simulate the model and return a simulation summary object:

In [ ]:
def simulate_model_test_rheobase(plot=False):
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['gl'] = soma(0.5).hh.gl
    sim_record['gnabar'] = soma(0.5).hh.gnabar
    sim_record['gkbar'] = soma(0.5).hh.gkbar
    
    sim_record['description'] = 'gna: %.2E S/cm^s; gk: %.2E S/cm^2' % (soma(0.5).hh.gnabar, soma(0.5).hh.gkbar)
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'], vm_threshold=-10.)
    sim_record['spike_times'] = spike_times
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(8, 3))
        axes[0].plot(sim_record['t'], sim_record['step_current_rec'], label='%.3f nA' % sim_record['stim_amp'])
        axes[0].set_xlabel('Time (ms)')
        axes[0].set_ylabel('Current (nA)')
        axes[0].set_title('Injected current')
        axes[1].plot(sim_record['t'], sim_record['soma_voltage'], label='%i spikes' % len(sim_record['spike_times']))
        axes[1].set_xlabel('Time (ms)')
        axes[1].set_ylabel('Voltage (mV)')
        axes[1].set_title('Soma membrane potential')
        axes[0].legend(loc='best', frameon=False)
        axes[1].legend(loc='best', frameon=False)
        fig.suptitle(sim_record['description'])
        fig.tight_layout(w_pad=0.9)
    
    return sim_record

Let's make sure our function works:

In [ ]:
step_current_stim.amp = 0.1
sim_record = simulate_model_test_rheobase(plot=True)

Now we want an objective function that will vary gna and gk, measure rheobase, and compute an objective based on how far away the measured rheobase is from a specified target value.

We also want to penalize spontaneous spiking that occurs outside the duration of the current injection.

In [ ]:
def get_rheobase_objective_error(x, targets, tolerances, bounds=None, sim_history=None, plot=False):
    
    # expect x to be an array of length 2, containing both a value of gna and a value of gk to test.
    # expect targets and tolerances to be dictionaries containing values for both 'rheobase' and 'spontaneous_spike_count'
    
    gnabar = x[0]
    gkbar = x[1]
    
    if bounds is not None:
        for i, (xmin, xmax) in enumerate(bounds):
            if not (xmin <= x[i] <= xmax):
                return 1e9
    
    soma(0.5).hh.gnabar = gnabar
    soma(0.5).hh.gkbar = gkbar
    
    # iterate over values of current injection until at least 1 spike occurs:
    for stim_amp in np.arange(0.025, 0.225, 0.025):
        step_current_stim.amp = stim_amp
        sim_record = simulate_model_test_rheobase(plot=plot)
        spike_times = sim_record['spike_times']
        if len(spike_times) > 0:
            break
    sim_record['rheobase'] = stim_amp
    sim_record['x'] = np.copy(x)
    
    # first construct a squared error term based on distance of stim_amp from the rheobase target:
    error = ((targets['rheobase'] - sim_record['rheobase']) / tolerances['rheobase']) ** 2.
    
    # now add an additional error based on spontaneous firing outside the duration of the step current injection:
    if len(spike_times) < 1:
        sim_record['spontaneous_spike_count'] = 0.
    else:
        # '&' is like 'and' for elemarrays, and '|' is like 'or' for arrays
        spont_spike_times = np.where((spike_times < step_current_stim.delay) | (spike_times > step_current_stim.delay + step_current_stim.dur))[0]
        sim_record['spontaneous_spike_count'] = len(spont_spike_times)
    
    error += ((targets['spontaneous_spike_count'] - sim_record['spontaneous_spike_count']) / tolerances['spontaneous_spike_count']) ** 2.
    
    sim_record['rheobase_error'] = error
    
    if sim_history is not None:
        sim_history.append(sim_record)
    
    return error

Let's make sure our objective function works. 

Now we need to define our targets and tolerances dictionaries, too.

In [ ]:
targets = {'rheobase': 0.1, 'spontaneous_spike_count': 0.}
tolerances = {'rheobase': 0.001, 'spontaneous_spike_count': 0.01}
bounds = [(gnabar_init / 100., gnabar_init * 10.), (gkbar_init / 100., gkbar_init * 10.)]
x0 = [gnabar_init, gkbar_init]

get_rheobase_objective_error(x0, targets, tolerances, bounds, plot=True)

Let's intentionally pass in values that are out of bounds to test our implementation of bound constraints:

In [ ]:
bounds = [(gnabar_init / 100., gnabar_init * 20.), (gkbar_init / 100., gkbar_init * 20.)]

get_rheobase_objective_error([gnabar_init * 100., gkbar_init * 100.], targets, tolerances, bounds)

Let's test it out with scipy.optimize.minimize!

First we will use the same gradient-based algorithm used above for optimization of input resistance.

In [ ]:
sim_history = []
x0 = [gnabar_init, gkbar_init]
result = minimize(get_rheobase_objective_error, x0, method='L-BFGS-B', bounds=bounds, options={'disp': True}, args=(targets, tolerances, bounds, sim_history))

In [ ]:
print(result)

Uh oh. It looks like it's still stuck at the initial conditions.

Let's look at the values that it explored:

In [ ]:
rheobase_history = []
spont_spike_count_history = []
gna_history = []
gk_history = []
error_history = []

for sim_record in sim_history:
    rheobase_history.append(sim_record['rheobase'])
    spont_spike_count_history.append(sim_record['spontaneous_spike_count'])
    gna_history.append(sim_record['x'][0])
    gk_history.append(sim_record['x'][1])
    error_history.append(sim_record['rheobase_error'])

fig, axes = plt.subplots(3, 2)
axes[0][0].scatter(gna_history, rheobase_history)
axes[0][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[0][0].set_ylabel('Rheobase (nA)')

axes[0][1].scatter(gk_history, rheobase_history)
axes[0][1].set_xlabel('K channel conductance (S/cm^2)')
axes[0][1].set_ylabel('Rheobase (nA)')

axes[1][0].scatter(gna_history, spont_spike_count_history)
axes[1][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[1][0].set_ylabel('Spontaneous spike count')

axes[1][1].scatter(gk_history, spont_spike_count_history)
axes[1][1].set_xlabel('K channel conductance (S/cm^2)')
axes[1][1].set_ylabel('Spontaneous spike count')

axes[2][0].scatter(gna_history, error_history)
axes[2][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[2][0].set_ylabel('Rheobase objective error')

axes[2][1].scatter(gk_history, error_history)
axes[2][1].set_xlabel('K channel conductance (S/cm^2)')
axes[2][1].set_ylabel('Rheobase objective error')

fig.tight_layout()

Last week we had success using another local search method that does not rely on gradient computation - the simplex algorithm. Let's see if that can help here:

In [ ]:
sim_history = []
x0 = [gnabar_init, gkbar_init]
result = minimize(get_rheobase_objective_error, x0, method='Nelder-Mead', options={'disp': True}, args=(targets, tolerances, bounds, sim_history))

In [ ]:
print(result)

In [ ]:
rheobase_history = []
spont_spike_count_history = []
gna_history = []
gk_history = []
error_history = []

for sim_record in sim_history:
    rheobase_history.append(sim_record['rheobase'])
    spont_spike_count_history.append(sim_record['spontaneous_spike_count'])
    gna_history.append(sim_record['x'][0])
    gk_history.append(sim_record['x'][1])
    error_history.append(sim_record['rheobase_error'])

fig, axes = plt.subplots(3, 2)
axes[0][0].scatter(gna_history, rheobase_history)
axes[0][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[0][0].set_ylabel('Rheobase (nA)')

axes[0][1].scatter(gk_history, rheobase_history)
axes[0][1].set_xlabel('K channel conductance (S/cm^2)')
axes[0][1].set_ylabel('Rheobase (nA)')

axes[1][0].scatter(gna_history, spont_spike_count_history)
axes[1][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[1][0].set_ylabel('Spontaneous spike count')

axes[1][1].scatter(gk_history, spont_spike_count_history)
axes[1][1].set_xlabel('K channel conductance (S/cm^2)')
axes[1][1].set_ylabel('Spontaneous spike count')

axes[2][0].scatter(gna_history, error_history)
axes[2][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[2][0].set_ylabel('Rheobase objective error')

axes[2][1].scatter(gk_history, error_history)
axes[2][1].set_xlabel('K channel conductance (S/cm^2)')
axes[2][1].set_ylabel('Rheobase objective error')

fig.tight_layout()

Despite testing more parameter values, the simplex algorithm performed a very local search. It never encountered any nearby parameters that altered the error landscape.

Let's try another algorithm, called "simulated annealing" that samples more widely within bounds:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.dual_annealing.html#scipy.optimize.dual_annealing

In [ ]:
from scipy.optimize import dual_annealing

sim_history = []
x0 = [gnabar_init, gkbar_init]
result = dual_annealing(get_rheobase_objective_error, x0=x0, no_local_search=True, seed=0, bounds=bounds, maxiter=100,
                      args=(targets, tolerances, bounds, sim_history))

In [ ]:
print(result)

This algorithm found a solution. Let's see how it explored the parameter space:

In [ ]:
rheobase_history = []
spont_spike_count_history = []
gna_history = []
gk_history = []
error_history = []

for sim_record in sim_history:
    rheobase_history.append(sim_record['rheobase'])
    spont_spike_count_history.append(sim_record['spontaneous_spike_count'])
    gna_history.append(sim_record['x'][0])
    gk_history.append(sim_record['x'][1])
    error_history.append(sim_record['rheobase_error'])

fig, axes = plt.subplots(3, 2)
axes[0][0].scatter(gna_history, rheobase_history)
axes[0][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[0][0].set_ylabel('Rheobase (nA)')

axes[0][1].scatter(gk_history, rheobase_history)
axes[0][1].set_xlabel('K channel conductance (S/cm^2)')
axes[0][1].set_ylabel('Rheobase (nA)')

axes[1][0].scatter(gna_history, spont_spike_count_history)
axes[1][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[1][0].set_ylabel('Spontaneous spike count')

axes[1][1].scatter(gk_history, spont_spike_count_history)
axes[1][1].set_xlabel('K channel conductance (S/cm^2)')
axes[1][1].set_ylabel('Spontaneous spike count')

axes[2][0].scatter(gna_history, error_history)
axes[2][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[2][0].set_ylabel('Rheobase objective error')

axes[2][1].scatter(gk_history, error_history)
axes[2][1].set_xlabel('K channel conductance (S/cm^2)')
axes[2][1].set_ylabel('Rheobase objective error')

fig.tight_layout()

Let's look at the traces:

In [ ]:
get_rheobase_objective_error(result.x, targets, tolerances, bounds, plot=True)

Now, ideally increasing the amount of injected current would increase the number of output spikes. Let's check:

In [ ]:
gnabar_new = result.x[0]
gkbar_new = result.x[1]

soma(0.5).hh.gnabar = gnabar_new
soma(0.5).hh.gkbar = gkbar_new

We can re-use our get_firing_rate function from last week:

In [ ]:
def get_firing_rate(spike_times, start, stop):
    duration = (stop - start) / 1000.  # convert to seconds
    spike_count = len(np.where((spike_times >= start) & (spike_times < stop))[0])
    return spike_count / duration  # in Hz

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for stim_amp in np.arange(0.1, 0.6, 0.1):
    sim_record = {}
    step_current_stim.amp = stim_amp
    sim_record['stim_amp'] = stim_amp
    sim_record['gl'] = soma(0.5).hh.gl
    sim_record['gnabar'] = soma(0.5).hh.gnabar
    sim_record['gkbar'] = soma(0.5).hh.gkbar
    
    sim_record['description'] = '%.3f nA' % stim_amp
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'])
    firing_rate = get_firing_rate(spike_times, step_current_stim.delay, step_current_stim.delay + step_current_stim.dur)
    sim_record['firing_rate'] = firing_rate
    
    axes[0].plot(sim_record['t'], sim_record['step_current_rec'], label=sim_record['description'])
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(sim_record['t'], sim_record['soma_voltage'], label='Firing rate: %.1f Hz' % sim_record['firing_rate'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

Hmm - this cell is still not very excitable. We optimized rheobase, but we did not attempt to optimize the slope of the f-I curve.

Let's attempt to add this additional constraint:

In [ ]:
targets = {'rheobase': 0.1, 'spontaneous_spike_count': 0., 'f_I_slope': 50.}  # f_I_slope units are Hz/nA
tolerances = {'rheobase': 0.001, 'spontaneous_spike_count': 0.01, 'f_I_slope': 1.}
bounds = [(gnabar_init / 100., gnabar_init * 20.), (gkbar_init / 100., gkbar_init * 20.)]

In [ ]:
def get_rheobase_objective_error_2(x, targets, tolerances, bounds=None, sim_history=None, plot=False):
    
    # expect x to be an array of length 2, containing both a value of gna and a value of gk to test.
    # expect targets and tolerances to be dictionaries containing values for 'rheobase', 'spontaneous_spike_count', and 'f_I_slope'
    
    gnabar = x[0]
    gkbar = x[1]
    
    if bounds is not None:
        for i, (xmin, xmax) in enumerate(bounds):
            if not (xmin <= x[i] <= xmax):
                return 1e9
    
    soma(0.5).hh.gnabar = gnabar
    soma(0.5).hh.gkbar = gkbar
    
    # iterate over values of current injection until at least 1 spike occurs:
    for stim_amp in np.arange(0.025, 0.225, 0.025):
        step_current_stim.amp = stim_amp
        sim_record = simulate_model_test_rheobase(plot=plot)
        spike_times = sim_record['spike_times']
        if len(spike_times) > 0:
            break
    sim_record['rheobase'] = stim_amp
    sim_record['x'] = np.copy(x)
    
    # first construct a squared error term based on distance of stim_amp from the rheobase target:
    error = ((targets['rheobase'] - sim_record['rheobase']) / tolerances['rheobase']) ** 2.
    
    # now add an additional error based on spontaneous firing outside the duration of the step current injection:
    start = step_current_stim.delay
    stop = step_current_stim.delay + step_current_stim.dur
    if len(spike_times) < 1:
        sim_record['spontaneous_spike_count'] = 0.
    else:
        # '&' is like 'and' for elemarrays, and '|' is like 'or' for arrays
        spont_spike_times = np.where((spike_times < start) | (spike_times > stop))[0]
        sim_record['spontaneous_spike_count'] = len(spont_spike_times)
    
    error += ((targets['spontaneous_spike_count'] - sim_record['spontaneous_spike_count']) / tolerances['spontaneous_spike_count']) ** 2.
    
    # now add an additional error that based on the slope of the f-I curve:
    sim_record['firing_rate_at_rheobase'] = get_firing_rate(spike_times, start, stop)
    delta_stim_amp = 0.3
    step_current_stim.amp = stim_amp + delta_stim_amp
    sim_record_2 = simulate_model_test_rheobase(plot=plot)
    spike_times_2 = sim_record_2['spike_times']
    firing_rate_2 = get_firing_rate(spike_times_2, start, stop)
    delta_firing_rate = firing_rate_2 - sim_record['firing_rate_at_rheobase']
    sim_record['f_I_slope'] = delta_firing_rate / delta_stim_amp
    
    error += ((targets['f_I_slope'] - sim_record['f_I_slope']) / tolerances['f_I_slope']) ** 2.
    
    sim_record['rheobase_error'] = error
    
    if sim_history is not None:
        sim_history.append(sim_record)
    
    return error

Let's make sure our new objective function works:

In [ ]:
x0 = [gnabar_new, gkbar_new]
get_rheobase_objective_error_2(x0, targets, tolerances, bounds, plot=True)

Now let's try the simulated annealing algorithm on this new objective function:

In [ ]:
sim_history = []
x0 = [gnabar_init, gkbar_init]
result = dual_annealing(get_rheobase_objective_error_2, x0=x0, local_search_options={'method': 'Nelder-Mead'}, 
                        seed=1, bounds=bounds, maxiter=200, args=(targets, tolerances, bounds, sim_history))

In [ ]:
print(result)

In [ ]:
get_rheobase_objective_error_2(result.x, targets, tolerances, bounds, plot=True)

In [ ]:
gnabar_new = result.x[0]
gkbar_new = result.x[1]

soma(0.5).hh.gnabar = gnabar_new
soma(0.5).hh.gkbar = gkbar_new

In [ ]:
rheobase_history = []
spont_spike_count_history = []
gna_history = []
gk_history = []
error_history = []
f_I_slope_history = []

for sim_record in sim_history:
    rheobase_history.append(sim_record['rheobase'])
    spont_spike_count_history.append(sim_record['spontaneous_spike_count'])
    gna_history.append(sim_record['x'][0])
    gk_history.append(sim_record['x'][1])
    f_I_slope_history.append(sim_record['f_I_slope'])
    error_history.append(sim_record['rheobase_error'])

fig, axes = plt.subplots(4, 2, figsize=(8, 10))
axes[0][0].scatter(gna_history, rheobase_history)
axes[0][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[0][0].set_ylabel('Rheobase (nA)')

axes[0][1].scatter(gk_history, rheobase_history)
axes[0][1].set_xlabel('K channel conductance (S/cm^2)')
axes[0][1].set_ylabel('Rheobase (nA)')

axes[1][0].scatter(gna_history, spont_spike_count_history)
axes[1][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[1][0].set_ylabel('Spontaneous spike count')

axes[1][1].scatter(gk_history, spont_spike_count_history)
axes[1][1].set_xlabel('K channel conductance (S/cm^2)')
axes[1][1].set_ylabel('Spontaneous spike count')

axes[2][0].scatter(gna_history, f_I_slope_history)
axes[2][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[2][0].set_ylabel('f-I slope (Hz/nA)')

axes[2][1].scatter(gk_history, f_I_slope_history)
axes[2][1].set_xlabel('K channel conductance (S/cm^2)')
axes[2][1].set_ylabel('f-I slope (Hz/nA)')

axes[3][0].scatter(gna_history, error_history)
axes[3][0].set_xlabel('Na channel conductance (S/cm^2)')
axes[3][0].set_ylabel('Rheobase objective error')

axes[3][1].scatter(gk_history, error_history)
axes[3][1].set_xlabel('K channel conductance (S/cm^2)')
axes[3][1].set_ylabel('Rheobase objective error')

fig.tight_layout()

## 2. Discuss papers from last week focusing on filtering of synaptic signals from dendrites to soma

- Magee, J., Cook, E. (2000). Somatic EPSP amplitude is independent of synapse location in hippocampal pyramidal neurons Nature Neuroscience  3(9), 895-903. https://dx.doi.org/10.1038/78800
- Magee, J. Dendritic Ih normalizes temporal summation in hippocampal CA1 neurons. Nat Neurosci 2, 508–514 (1999). https://doi.org/10.1038/9158

## 3. Explore attenuation of signals from dendrite to soma

### We are ready to create a dendritic section and connect it to the soma compartment!

In [ ]:
dend = h.Section()
dend.L = 300.
dend.diam = 1.5
dend.nseg = 9  # Split it into 9 parts to increase the spatial resolution of the simulation
dend.Ra = 100.

# parent section, parent location, child location
dend.connect(soma, 1., 0.)

dend.insert('hh')

# Let's just simulate passive dendrites without Na or K channels for today.
# Here is syntax for setting a membrane density parameter for all segments of a section at once:
dend.gnabar_hh = 0.
dend.gkbar_hh = 0.
dend.gl_hh = gl_new

Let's set up multiple voltage recordings from the dendritic section:

In [ ]:
dend_voltage_rec_list = []
for seg in dend:
    dend_rec = h.Vector()
    dend_rec.record(seg._ref_v)  # record the voltage across the membrane in a segment
    dend_voltage_rec_list.append(dend_rec)

Let's make sure our recordings work with a step current injection into the soma:

In [ ]:
step_current_stim.amp = -0.1
h.run()

plt.figure()
plt.plot(t, soma_voltage, label='Soma')
for dend_rec, seg in zip(dend_voltage_rec_list, dend):
    distance = dend.L * seg.x
    plt.plot(t, dend_rec, label='Dend: %.1f um' % distance)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')
plt.legend(loc='best', frameon=False)

BTW, what has adding a dendrite done to somatic input resistance?

In [ ]:
# Let's forget about spiking conductances for now:
soma(0.5).hh.gnabar = 0.
soma(0.5).hh.gkbar = 0.
step_current_stim.amp = -0.1
sim_record = simulate_model_test_input_resistance(plot=True)

Let's try compensating by modifying somatic and dendritic leak conductance. Input resistance is typically higher in dendrites due to the lower diameter and reduced surface area.

In [ ]:
soma(0.5).hh.gl = gl_new * 1.06
dend.gl_hh = soma(0.5).hh.gl
sim_record = simulate_model_test_input_resistance(plot=True)
print('Somatic gl: %.3E; dendritic gl: %.3E' % (soma(0.5).hh.gl, dend.gl_hh))

What about excitability?

In [ ]:
get_rheobase_objective_error_2([gnabar_new, gkbar_new], targets, tolerances, plot=True)

### Clearly, extra membrane and extra conductances in a dendritic compartment change the excitability of the soma. 

Properties like input resistance and f-I slope would have to be tuned with the full morphology pre-constructed.

Let's turn off the Na and K channels for now to look at the passive filtering properties of the dendritic cable.

In [ ]:
soma(0.5).hh.gnabar = 0.
soma(0.5).hh.gkbar = 0.

### Let's turn off our somatic current injection and add one to the end of the dendrite:

In [ ]:
step_current_stim.amp = 0.

dend_current_stim = h.IClamp(dend(1.))

### But instead of injecting a square step current, let's construct a waveform to inject:

In [ ]:
t_array = np.array(t)
rise_and_decay = np.exp(-(t_array - 200.) / 10.) - np.exp(-(t_array - 200.) / 0.5)
rise_and_decay[np.where(t_array < 200.)[0]] = 0.
rise_and_decay /= np.max(rise_and_decay)
plt.figure()
plt.plot(t_array, rise_and_decay)

In [ ]:
step_current_stim.amp = 0.
dend_current_stim.dur = 1e9  # the waveform will determine the duration
dend_current_stim.delay = 0.  # the baseline is now built in to the waveform

dend_stim_waveform_amp = 0.02 # nA
dend_stim_amp_vector = h.Vector(dend_stim_waveform_amp * rise_and_decay)
dend_stim_t_vector = h.Vector(t_array)
dend_stim_amp_vector.play(dend_current_stim._ref_amp, dend_stim_t_vector)

### Let's look at the amplitude and kinetics of the response, to compare to the Magee papers.

In [ ]:
h.run()

fig, axes = plt.subplots(1, 2, sharex=True)
t_array = np.array(t)
soma_voltage_array = np.array(soma_voltage)
baseline_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
EPSP_indexes = np.where((t_array >= 200.) & (t_array < 250.))[0]
baseline = np.mean(soma_voltage_array[baseline_indexes])
soma_voltage_array -= baseline
axes[0].plot(t, soma_voltage_array, label='Soma')
axes[1].plot(t, soma_voltage_array / np.max(soma_voltage_array[EPSP_indexes]))

for dend_voltage, seg in zip(dend_voltage_rec_list, dend):
    distance = dend.L * seg.x
    dend_voltage_array = np.array(dend_voltage)
    baseline = np.mean(dend_voltage_array[baseline_indexes])
    dend_voltage_array -= baseline
    axes[0].plot(t, dend_voltage_array, label='Dend (%.2f um)' % distance)
    axes[1].plot(t, dend_voltage_array / np.max(dend_voltage_array[EPSP_indexes]))

axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_ylabel('Normalized amplitude (mV)')
axes[0].set_xlim((190., 250.))
axes[0].set_ylim((-0.5, axes[0].get_ylim()[1]))
axes[1].set_ylim((-0.5, axes[1].get_ylim()[1]))

fig.tight_layout()

## 4. Challenge - Reproduce Figures 2b, 3a, and 3c from Magee & Cook

 - Iterate over dendritic segments.
 - Move the dendritic stimulus to the new site.
 - Record local dendritic and remote somatic voltage.
 - Save the data into a sim_history data structure.
 - Plot EPSP amplitude vs. distance of stimulation site from soma for both recordings from soma and dendrite.
 - Plot EPSP rise time vs. distance of stimulation site from soma for both recordings from soma and dendrite.